### Exercícios - With Column Parte 1

In [7]:
#!hdfs dfs -cat /user/marco/data/juros_selic/juros_selic

In [3]:
from pyspark.sql.functions import *

In [9]:
# 1. Criar um dataframe para ler o arquivo no HDFS /user/<nome/data/juros_selic/juros_selic
juros = spark.read.csv("/user/marco/data/juros_selic/juros_selic", sep=";", header="true")

In [10]:
# 2. Alterar o formato do campo data para “MM/dd/yy
juros.show(20)

+----------+-----+
|      data|valor|
+----------+-----+
|01/06/1986| 1,27|
|01/07/1986| 1,95|
|01/08/1986| 2,57|
|01/09/1986| 2,94|
|01/10/1986| 1,96|
|01/11/1986| 2,37|
|01/12/1986| 5,47|
|01/01/1987|11,00|
|01/02/1987|19,61|
|01/03/1987|11,95|
|01/04/1987|15,30|
|01/05/1987|24,63|
|01/06/1987|18,02|
|01/07/1987| 8,91|
|01/08/1987| 8,09|
|01/09/1987| 7,99|
|01/10/1987| 9,45|
|01/11/1987|12,92|
|01/12/1987|14,38|
|01/01/1988|16,78|
+----------+-----+
only showing top 20 rows



In [11]:
# Códigos separados
juros_date_americano_unix = juros.withColumn("data", unix_timestamp(col("data"), "dd/MM/yyyy"))
juros_date_americano = juros_date_americano_unix.withColumn("data", from_unixtime("data", "MM/dd/yyyy"))

In [12]:
# Códigos em apenas uma linha
juros_date_americano = juros.withColumn("data", from_unixtime(unix_timestamp(col("data"), "dd/MM/yyyy"), "MM/dd/yyyy"))
juros_date_americano.show(3)

+----------+-----+
|      data|valor|
+----------+-----+
|06/01/1986| 1,27|
|07/01/1986| 1,95|
|08/01/1986| 2,57|
+----------+-----+
only showing top 3 rows



In [13]:
# 3.Com uso da função from_unixtime crie o campo “ano_unix”, com a informação do ano do campo data
juros_unixtime = juros_date_americano.withColumn("ano_unix", from_unixtime(unix_timestamp(col("data"), "dd/MM/yyyy"), "yyyy"))
juros_unixtime.show(3)

+----------+-----+--------+
|      data|valor|ano_unix|
+----------+-----+--------+
|06/01/1986| 1,27|    1986|
|07/01/1986| 1,95|    1986|
|08/01/1986| 2,57|    1986|
+----------+-----+--------+
only showing top 3 rows



In [14]:
# 4.Com uso de substring crie o campo “ano_str”, com a informação do ano do campo data
juros_substring = juros_unixtime.withColumn("ano_str", substring(col("data"), 7, 4))
juros_substring.show(3)

+----------+-----+--------+-------+
|      data|valor|ano_unix|ano_str|
+----------+-----+--------+-------+
|06/01/1986| 1,27|    1986|   1986|
|07/01/1986| 1,95|    1986|   1986|
|08/01/1986| 2,57|    1986|   1986|
+----------+-----+--------+-------+
only showing top 3 rows



In [15]:
# 5.Com uso da função split crie o campo “ano_str”, com a informação do ano do campo data
juros_split = juros_substring.withColumn("ano_split", split(col("data"), "/").getItem(2))
juros_split.show(3)

+----------+-----+--------+-------+---------+
|      data|valor|ano_unix|ano_str|ano_split|
+----------+-----+--------+-------+---------+
|06/01/1986| 1,27|    1986|   1986|     1986|
|07/01/1986| 1,95|    1986|   1986|     1986|
|08/01/1986| 2,57|    1986|   1986|     1986|
+----------+-----+--------+-------+---------+
only showing top 3 rows



In [17]:
# 6.Salvar no hdfs / user / <nome> /juros_selic_ americano no formato CSV, incluindo o cabeçalho
juros_split.write.csv("/user/marco/juros_selic_americano", header="true")

AnalysisException: 'path hdfs://namenode:8020/user/marco/juros_selic_americano already exists.;'

In [18]:
# Checar se realmente salvou
!hdfs dfs -ls /user/marco/juros_selic_americano

Found 2 items
-rw-r--r--   2 root supergroup          0 2021-06-27 20:34 /user/marco/juros_selic_americano/_SUCCESS
-rw-r--r--   2 root supergroup      13086 2021-06-27 20:34 /user/marco/juros_selic_americano/part-00000-b9080cb9-4cc1-4db0-90a6-3d8a8469b205-c000.csv


In [19]:
# Visualizar dados salvos
!hdfs dfs -cat /user/marco/juros_selic_americano/part-00000-b9080cb9-4cc1-4db0-90a6-3d8a8469b205-c000.csv

data,valor,ano_unix,ano_str,ano_split
06/01/1986,"1,27",1986,1986,1986
07/01/1986,"1,95",1986,1986,1986
08/01/1986,"2,57",1986,1986,1986
09/01/1986,"2,94",1986,1986,1986
10/01/1986,"1,96",1986,1986,1986
11/01/1986,"2,37",1986,1986,1986
12/01/1986,"5,47",1986,1986,1986
01/01/1987,"11,00",1987,1987,1987
02/01/1987,"19,61",1987,1987,1987
03/01/1987,"11,95",1987,1987,1987
04/01/1987,"15,30",1987,1987,1987
05/01/1987,"24,63",1987,1987,1987
06/01/1987,"18,02",1987,1987,1987
07/01/1987,"8,91",1987,1987,1987
08/01/1987,"8,09",1987,1987,1987
09/01/1987,"7,99",1987,1987,1987
10/01/1987,"9,45",1987,1987,1987
11/01/1987,"12,92",1987,1987,1987
12/01/1987,"14,38",1987,1987,1987
01/01/1988,"16,78",1988,1988,1988
02/01/1988,"18,35",1988,1988,1988
03/01/1988,"16,59",1988,1988,1988
04/01/1988,"20,25",1988,1988,1988
05/01/1988,"18,65",1988,1988,1988
06/01/1988,"20,17",1988,1988,1988
07/01/1988,"24,69",1988,1988,1988
08/01/1988,"22,63",1988,1988,1988
09/01/1988,"26,25",1988,1988,1988
10/01/1988,"29,79",19

### Exercícios - Whith Column Parte 2

In [8]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [5]:
# 1. Criar um dataframe para ler o arquivo no HDFS /user/<nome/data/juros_selic/juros_selic
juros_selic = spark.read.csv("/user/marco/data/juros_selic/juros_selic", header="true", sep=";")
juros_selic.show(3)

+----------+-----+
|      data|valor|
+----------+-----+
|01/06/1986| 1,27|
|01/07/1986| 1,95|
|01/08/1986| 2,57|
+----------+-----+
only showing top 3 rows



In [11]:
# 2.Agrupar todas as datas pelo ano em ordem decrescente e salvar a quantidade de meses ocorridos, o valor médio, mínimo e máximo do campo valor
juros_ano = juros_selic.withColumn("ano", split(col("data"), "/").getItem(2))
juros_valor = juros_ano.withColumn("valor", regexp_replace(col("valor"), "\,", "\.").cast(FloatType()))
juros_valor.printSchema()
juros_valor.show(3)

root
 |-- data: string (nullable = true)
 |-- valor: float (nullable = true)
 |-- ano: string (nullable = true)

+----------+-----+----+
|      data|valor| ano|
+----------+-----+----+
|01/06/1986| 1.27|1986|
|01/07/1986| 1.95|1986|
|01/08/1986| 2.57|1986|
+----------+-----+----+
only showing top 3 rows



In [20]:
juros_relatorio = juros_valor.groupBy("ano").agg(count("ano").alias("Meses"),format_number(avg("valor"), 2).alias("Valor Médio"), min("valor").alias("Valor Mínimo"), max("valor").alias("Valor Máximo")).sort(desc("ano"))

In [21]:
# 3.Salvar no hdfs user /< relatorioAnual com compressão zlib e formato orc
juros_relatorio.write.orc("/user/marco/relatorio_anual", compression="zlib")

In [22]:
!hdfs dfs -ls /user/marco/relatorio_anual

Found 35 items
-rw-r--r--   2 root supergroup          0 2021-06-28 17:57 /user/marco/relatorio_anual/_SUCCESS
-rw-r--r--   2 root supergroup        582 2021-06-28 17:57 /user/marco/relatorio_anual/part-00000-25a99573-bab5-4395-8222-d0eb6d54b84b-c000.zlib.orc
-rw-r--r--   2 root supergroup        582 2021-06-28 17:57 /user/marco/relatorio_anual/part-00001-25a99573-bab5-4395-8222-d0eb6d54b84b-c000.zlib.orc
-rw-r--r--   2 root supergroup        585 2021-06-28 17:57 /user/marco/relatorio_anual/part-00002-25a99573-bab5-4395-8222-d0eb6d54b84b-c000.zlib.orc
-rw-r--r--   2 root supergroup        572 2021-06-28 17:57 /user/marco/relatorio_anual/part-00003-25a99573-bab5-4395-8222-d0eb6d54b84b-c000.zlib.orc
-rw-r--r--   2 root supergroup        583 2021-06-28 17:57 /user/marco/relatorio_anual/part-00004-25a99573-bab5-4395-8222-d0eb6d54b84b-c000.zlib.orc
-rw-r--r--   2 root supergroup        582 2021-06-28 17:57 /user/marco/relatorio_anual/part-00005-25a99573-bab5-4395-8222-d0eb6d54b84b-c000.zlib